In [1]:
import random
import numpy as np
import json
from tqdm import tqdm
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist, squareform
from scipy.optimize import linear_sum_assignment
from collections import Counter
import time
import os
import pandas as pd
from scipy.signal import correlate
np. set_printoptions(threshold=np. inf)

In [2]:
date = 'Sept27th-'
filename = ''.join((date,'scan_stats.json'))
filename

'Sept27th-scan_stats.json'

In [3]:
import scan
import simulate_data

In [12]:
default_params = {
    'M':8,
    'N':40,
    'D':51,
    'T':1000,
    'seed':0,
    'num_SM_events':16,
    'SM_total_spikes':10,
    'noise':100
}
scan_dict = {
    'M':[1,4,8,16,32],
    'N':[10,20,40,80,120],
    'D':[11,31,51,71,101],
    'num_SM_events':[2,4,8,16,32],
    'SM_total_spikes':[3,5,10,20,50],
    'noise':[0,50,100,500,1000]
}

In [13]:
def get_acc(matrix_x,matrix_y):
    # Calculate cross-correlation matrix
    cc = np.zeros((matrix_x.shape[2], matrix_y.shape[2]))

    for x_channel_idx in range(matrix_x.shape[2]):
        for y_channel_idx in range(matrix_y.shape[2]):
            cc[x_channel_idx, y_channel_idx], _ = max_overlap(matrix_x[...,x_channel_idx],matrix_y[...,y_channel_idx])
    SM_acc = np.max(cc,axis=1)
    return SM_acc, cc

In [14]:
def max_overlap(image, kernel):
    result = np.zeros((image.shape[1]+kernel.shape[1]-1))
    for n in range(image.shape[0]):
        result += correlate(image[n,:], kernel[n,:], mode = 'full')
    return np.max(result)/max(np.sum(image),np.sum(kernel)), np.argmax(result)

In [15]:
def get_imgs(K_dense, pattern_template):
    if len(pattern_template) == 0:
        print('FAIL')
        return pattern_template, None
    
    win_size = (K_dense.shape[0],1+max([max(k[:,0]) for k in pattern_template]))
    pattern_img = np.zeros((len(pattern_template),*win_size))
    for p,pattern in enumerate(pattern_template):
        for (i,j) in pattern:
            pattern_img[p,j,i] = 1
            
    pattern_img = pattern_img.transpose((1,2,0))
    
    return pattern_template, pattern_img

In [16]:
from scipy.signal import correlate
def get_acc(ground_truths,detected_patterns):
    # Calculate cross-correlation matrix
    cross_corr_matrix = np.zeros((ground_truths.shape[2], detected_patterns.shape[2]))
    SM_acc = np.zeros((ground_truths.shape[2]))
    
    if len(detected_patterns) == 0:
        return SM_acc, cross_corr_matrix
    
    for ground_truths_idx in range(ground_truths.shape[2]):
        for detected_patterns_idx in range(detected_patterns.shape[2]):
            cross_corr = np.zeros((ground_truths.shape[1]+detected_patterns.shape[1]-1))
            for n in range(ground_truths.shape[0]):
                cross_corr += correlate(ground_truths[n, :, ground_truths_idx], detected_patterns[n, :, detected_patterns_idx], mode='full')
            max_corr = np.max(cross_corr) / max(np.sum(ground_truths[...,ground_truths_idx]),np.sum(detected_patterns[...,detected_patterns_idx]))
            cross_corr_matrix[ground_truths_idx, detected_patterns_idx] = max_corr
#     print(cross_corr_matrix)
#     print( np.sum(ground_truths[...,ground_truths_idx]))
    SM_acc = np.max(cross_corr_matrix,axis=1)
    return SM_acc, cross_corr_matrix

In [17]:
def check_ground_truth(pattern_template, K_dense):
    _, pattern_img = get_imgs(K_dense, pattern_template)
    SM_acc, cc = get_acc(K_dense, pattern_img)
    return SM_acc, cc, pattern_img
    

In [18]:
param_combinations = []
for param_name, param_values in scan_dict.items():
    for param_value in param_values:
        params = default_params.copy()
        params[param_name] = param_value
        param_combinations.append(params)

In [19]:
len(param_combinations)

30

In [20]:
import numpy as np
import random
import os
import pandas as pd
import json
from tqdm import tqdm

# Define default parameters and scan values
default_params = {
    'M': 8,
    'N': 40,
    'D': 51,
    'T': 1000,
    'seed': 0,
    'num_SM_events': 8,
    'SM_total_spikes': 10,
    'noise': 100
}

scan_dict = {
    'M': [1, 4, 8, 16, 32],
    'N': [10, 20, 40, 80, 120],
    'D': [11, 31, 51, 71, 101], 
    'num_SM_events': [2, 4, 8, 16, 32],
    'SM_total_spikes': [3, 5, 10, 20, 50],
    'noise': [0, 50, 100, 500, 1000]
}

# Generate all parameter combinations
param_combinations = []
for param_name, param_values in scan_dict.items():
    for param_value in param_values:
        params = default_params.copy()
        params[param_name] = param_value
        param_combinations.append(params)

num_samples = len(param_combinations)
results = []

# Iterate through parameter combinations
for idx, params in tqdm(enumerate(param_combinations)):

    if os.path.isfile(filename):
        df = pd.read_json(filename)
        with open(filename, 'r') as results_file:
            results = json.load(results_file)
    print(params)
    start = time.time()
    A_dense, A_sparse, B_dense, B_sparse, K_dense, K_sparse = simulate_data.generate_synthetic_data(params)
    pattern_template, sublist_keys_filt, window_time, cluster_time, sequence_time = scan.scan_raster(A_sparse[1], A_sparse[0], window_dim=params['D'])
    SM_acc, cc, _ = check_ground_truth(pattern_template, K_dense)
    end = time.time()
    result = {
        'M':params['M'],
        'N':params['N'],
        'D':params['D'],
        'T':params['T'],
        'num_SM_events':params['num_SM_events'],
        'SM_total_spikes':params['SM_total_spikes'],
        'noise':params['noise'],
        'window_time': window_time,
        'cluster_time': cluster_time,
        'sequence_time': sequence_time,
        'total_time': end-start,
        'total_patterns':len(pattern_template),
        'SM_acc':SM_acc.tolist()
    }

    results.append(result)

    with open(filename, 'w') as results_file:
        json.dump(results, results_file, indent=4)


0it [00:00, ?it/s]

{'M': 1, 'N': 40, 'D': 51, 'T': 1000, 'seed': 0, 'num_SM_events': 8, 'SM_total_spikes': 10, 'noise': 100}
180 Windows


1it [00:01,  1.16s/it]

5 patterns found...s... 80% 10.01 | opt_cutoff - 1.09 | most_detections - 7etections - 7
{'M': 4, 'N': 40, 'D': 51, 'T': 1000, 'seed': 0, 'num_SM_events': 8, 'SM_total_spikes': 10, 'noise': 100}
417 Windows
12 patterns found...... 92% 10.01 | opt_cutoff - 0.51 | most_detections - 7etections - 75


2it [00:03,  1.73s/it]

...

{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 0, 'num_SM_events': 8, 'SM_total_spikes': 10, 'noise': 500}
1131 Windows


7 patterns found...s... 86% 10.01 | opt_cutoff - 0.38 | most_detections - 8etections - 88_detections - 1


29it [03:28,  8.77s/it]

{'M': 8, 'N': 40, 'D': 51, 'T': 1000, 'seed': 0, 'num_SM_events': 8, 'SM_total_spikes': 10, 'noise': 1000}
1621 Windows
4 patterns found...s... 75% 10.01 | opt_cutoff - 0.3 | most_detections - 7etections - 771


30it [03:34,  7.15s/it]


In [21]:
df = pd.read_json(filename)

In [22]:
df

,M,N,D,T,num_SM_events,SM_total_spikes,noise,window_time,cluster_time,sequence_time,total_time,total_patterns,SM_acc
0,1,40,51,1000,8,10,100,0.037481,1.066340,0.002001,1.155551,5,[1.0]
1,4,40,51,1000,8,10,100,0.118966,1.577544,0.002994,2.098831,12,"[1.0, 1.0, 1.0, 1.0]"
2,8,40,51,1000,8,10,100,0.330320,2.029799,0.004990,3.223248,19,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
3,16,40,51,1000,8,10,100,1.256067,3.570322,0.007997,8.966395,20,"[1.0, 1.0, 0.769230769230769, 1.0, 1.0, 1.0, 1..."
4,32,40,51,1000,8,10,100,5.909603,5.974322,0.000944,12.210968,1,"[0.030303030303030002, 0.030303030303030002, 0..."
5,8,10,51,1000,8,10,100,0.672758,1.711184,0.004036,2.966406,13,"[1.0, 0.2, 0.769230769230769, 0.625, 1.0, 1.0,..."
6,8,20,51,1000,8,10,100,0.397215,1.672701,0.005984,2.790130,24,"[1.0, 0.6666666666666661, 1.0, 1.0, 1.0, 1.0, ..."
7,8,40,51,1000,8,10,100,0.315226,2.006551,0.004987,3.161595,19,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
8,8,80,51,1000,8,10,100,0.275037,2.564061,0.006979,4.513465,22,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
9,8,120,51,1000,8,10,100,0.267627,2.819906,0.005984,5.290575,21,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"


In [51]:
N = 1000
M = 10
D = 71
T = 1000
seed=0

num_SM_events = 10
SM_total_spikes = 150
noise = 1000

params = {
    'N':N,
    'M':M,
    'D':D,
    'T':T,
    'seed':seed,
    'num_SM_events':num_SM_events,
    'SM_total_spikes':SM_total_spikes,
    'noise':noise
}

start = time.time()
A_dense, A_sparse, B_dense, B_sparse, K_dense, K_sparse = simulate_data.generate_synthetic_data(params)
pattern_template, sublist_keys_filt, window_time, cluster_time, sequence_time = scan.scan_raster(A_sparse[1], A_sparse[0], window_dim=params['D'])
end = time.time()
print(end-start)

15867 Windows
38 patterns found...... 97% 10.01 | opt_cutoff - 0.38 | most_detections - 19tections - 199detections - 7


KeyboardInterrupt: 